In [184]:
# set env variable to json credential file of google cloud

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/karthikmahendra/Desktop/Salty/salty-trolls-cce24104ff9f.json'

In [185]:
from google.cloud import bigquery
import time
import pandas as pd
import numpy as np

In [186]:
# setup client
bq_client = bigquery.Client() 

In [187]:
#Query for pulling data of a username
QUERY = '''SELECT  * 
        FROM `bigquery-public-data.hacker_news.full_201510`
        WHERE `bigquery-public-data.hacker_news.full_201510`.by = "danmaz74"
        '''

In [188]:
df = bq_client.query(QUERY).to_dataframe()

In [189]:
df.shape

(1680, 13)

In [190]:
df.head()

,by,score,time,title,type,url,text,parent,deleted,dead,descendants,id,ranking
0,danmaz74,NaN,1386177239,None,comment,None,Don&#x27;t be silly yourself. What happens if ...,6847999.0,None,None,NaN,6848340,None
1,danmaz74,NaN,1338127924,None,comment,None,"This is what I was afraid of. By the way, with...",4030098.0,None,None,NaN,4030122,None
2,danmaz74,NaN,1324923786,None,comment,None,"Thank you for your suggestions.<p>The ""Executi...",3393433.0,None,None,NaN,3393485,None
3,danmaz74,NaN,1316963493,None,comment,None,G+ is still missing some form of tags/hashtags...,3035527.0,None,None,NaN,3035905,None
4,danmaz74,NaN,1419506664,None,comment,None,Yes I also thought the same thing... on the ot...,8796028.0,None,None,NaN,8796044,None


In [144]:
# import html,re

In [145]:
# def decoder(text):
#     """decode HTML text."""
#     unescaped = html.unescape(text)
#     return unescaped


# def kill_html(text):
#     """Remove html flags."""
#     cleanr = re.compile('<.*?>')
#     cleantext = re.sub(cleanr, ' ', text)
#     return cleantext


# def kill_url(text):
#     """Remove URL with regex"""
#     return ''.join(re.sub(r"http\S+", "", text))





In [146]:
# #df['cleaned_comment'] = np.zeros(df.shape[0])
# kill_url(kill_html(decoder()

In [191]:
# to get rid of the weird characters
import html
df['text'] = df['text'].apply(str)

In [192]:
df['text'] = df['text'].apply(lambda x: html.unescape(x))

In [193]:
import re
def remove_html_tags(text):
   """Remove html tags from a string"""
   clean = re.compile('<.*?>')
   return re.sub(clean, '', text)

In [194]:
df['text'] = df['text'].apply(lambda x: remove_html_tags(x))

In [195]:
df.head()

,by,score,time,title,type,url,text,parent,deleted,dead,descendants,id,ranking
0,danmaz74,NaN,1386177239,None,comment,None,Don't be silly yourself. What happens if you a...,6847999.0,None,None,NaN,6848340,None
1,danmaz74,NaN,1338127924,None,comment,None,"This is what I was afraid of. By the way, with...",4030098.0,None,None,NaN,4030122,None
2,danmaz74,NaN,1324923786,None,comment,None,"Thank you for your suggestions.The ""Executive ...",3393433.0,None,None,NaN,3393485,None
3,danmaz74,NaN,1316963493,None,comment,None,G+ is still missing some form of tags/hashtags...,3035527.0,None,None,NaN,3035905,None
4,danmaz74,NaN,1419506664,None,comment,None,Yes I also thought the same thing... on the ot...,8796028.0,None,None,NaN,8796044,None


In [110]:
# job_config = bigquery.QueryJobConfig()
# now=time.time()
# query_job=bq_client.query(QUERY,location='US')
# res=query_job.result()
# print('query took:',round(time.time()-now,2),'s')

In [111]:
# now=time.time()
# destination_uri = "gs://karthikkm028-ds/*hacker.csv"
# dataset_ref = bq_client.dataset("hacker_news", project='bigquery-public-data')
# table_ref = dataset_ref.table("full_201510")

# extract_job = bq_client.extract_table(
#     table_ref,
#     destination_uri)
# extract_job.result()  # Waits for job to complete
# print('create table and write to GCS took:',round(time.time()-now,2),'s')

In [112]:
# gcs = gcsfs.GCSFileSystem(project='bigquery-public-data') 
# files=gcs.glob(destination_uri)
# df = pd.concat([pd.read_csv('gs://'+f) for f in files], ignore_index=True)
# print('read csv took:',round(time.time()-now,2),'s')

In [208]:

df_drop = df.drop(columns= ['score','title','url','deleted','dead','descendants','ranking'])

In [209]:
df_drop.head()

,by,time,type,text,parent,id
0,danmaz74,1386177239,comment,Don't be silly yourself. What happens if you a...,6847999.0,6848340
1,danmaz74,1338127924,comment,"This is what I was afraid of. By the way, with...",4030098.0,4030122
2,danmaz74,1324923786,comment,"Thank you for your suggestions.The ""Executive ...",3393433.0,3393485
3,danmaz74,1316963493,comment,G+ is still missing some form of tags/hashtags...,3035527.0,3035905
4,danmaz74,1419506664,comment,Yes I also thought the same thing... on the ot...,8796028.0,8796044


In [210]:
df_drop.shape

(1680, 6)

In [211]:
df_drop.dtypes

by         object
time        int64
type       object
text       object
parent    float64
id          int64
dtype: object

In [212]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [213]:
sia.polarity_scores(df_drop['text'][0])

{'neg': 0.008, 'neu': 0.882, 'pos': 0.11, 'compound': 0.9248}

In [214]:
# df_drop['neg']=np.zeros(df_drop.shape[0])
# df_drop['pos']=np.zeros(df_drop.shape[0])
# # df_drop['neu']=np.zeros(df_drop.shape[0])

In [215]:
now=time.time()
df_drop['neg'] = df_drop['text'].apply(lambda x: sia.polarity_scores(x)['neg'])
df_drop['pos'] = df_drop['text'].apply(lambda x: sia.polarity_scores(x)['pos'])
print('sentiment cal took:',round(time.time()-now,2),'s')

sentiment cal took: 2.09 s


In [216]:
df_drop.head()

,by,time,type,text,parent,id,neg,pos
0,danmaz74,1386177239,comment,Don't be silly yourself. What happens if you a...,6847999.0,6848340,0.008,0.110
1,danmaz74,1338127924,comment,"This is what I was afraid of. By the way, with...",4030098.0,4030122,0.140,0.000
2,danmaz74,1324923786,comment,"Thank you for your suggestions.The ""Executive ...",3393433.0,3393485,0.036,0.169
3,danmaz74,1316963493,comment,G+ is still missing some form of tags/hashtags...,3035527.0,3035905,0.129,0.089
4,danmaz74,1419506664,comment,Yes I also thought the same thing... on the ot...,8796028.0,8796044,0.000,0.308


In [177]:
# # populate vader sentiments in additional cols
# now=time.time()
# for idx,x in df_drop['text'].iteritems():
#     #print (idx,x)
#     df_drop['neg'][idx] = sia.polarity_scores(x)['neg']
#     df_drop['pos'][idx] = sia.polarity_scores(x)['pos']
# #     df_drop['neu'][idx]= sia.polarity_scores(x)['neu']
# print('sentiment cal took:',round(time.time()-now,2),'s')

/Users/karthikmahendra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/karthikmahendra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


sentiment cal took: 101.31 s


In [217]:
# Set threshold of .5 for filtering positive and negative comments. drop comments with value 1 

df_neg = df_drop[(df_drop['neg']> 0.5) & (df_drop['neg'] !=1)]
df_pos = df_drop[(df_drop['pos']> 0.5) & (df_drop['pos'] !=1)]

In [218]:
from datetime import datetime

#convert unix time to datetime object with accuracy of secs

df_drop['time']=pd.to_datetime(df_drop['time'],unit='s')

In [219]:
# top 10 positive comments
df_pos = df_drop.head(10)

In [220]:
# # top 10 negative comments
df_neg = df_drop.head(10)

In [221]:
df_pos = df_pos.sort_values(['time'],ascending=False)
df_neg = df_neg.sort_values(['time'],ascending=False)

In [222]:
df_pos

,by,time,type,text,parent,id,neg,pos
4,danmaz74,2014-12-25 11:24:24,comment,Yes I also thought the same thing... on the ot...,8796028.0,8796044,0.000,0.308
6,danmaz74,2014-08-23 05:43:36,comment,In real life architecture the wall/surface is ...,8214599.0,8215119,0.193,0.000
0,danmaz74,2013-12-04 17:13:59,comment,Don't be silly yourself. What happens if you a...,6847999.0,6848340,0.008,0.110
8,danmaz74,2013-10-18 18:51:37,comment,This looks like a very good solution. If enoug...,6572920.0,6573252,0.000,0.469
7,danmaz74,2013-10-18 10:50:34,comment,"Lost had much higher production value, but B5 ...",6569359.0,6570689,0.074,0.255
5,danmaz74,2013-07-15 13:35:34,comment,"Come on, no serious economist gives credit to ...",6045121.0,6045388,0.056,0.202
1,danmaz74,2012-05-27 14:12:04,comment,"This is what I was afraid of. By the way, with...",4030098.0,4030122,0.140,0.000
2,danmaz74,2011-12-26 18:23:06,comment,"Thank you for your suggestions.The ""Executive ...",3393433.0,3393485,0.036,0.169
9,danmaz74,2011-10-13 09:15:39,comment,In my day - I'm 37 - I remember being VERY fre...,3106543.0,3106671,0.059,0.205
3,danmaz74,2011-09-25 15:11:33,comment,G+ is still missing some form of tags/hashtags...,3035527.0,3035905,0.129,0.089


In [223]:
df_neg

,by,time,type,text,parent,id,neg,pos
4,danmaz74,2014-12-25 11:24:24,comment,Yes I also thought the same thing... on the ot...,8796028.0,8796044,0.000,0.308
6,danmaz74,2014-08-23 05:43:36,comment,In real life architecture the wall/surface is ...,8214599.0,8215119,0.193,0.000
0,danmaz74,2013-12-04 17:13:59,comment,Don't be silly yourself. What happens if you a...,6847999.0,6848340,0.008,0.110
8,danmaz74,2013-10-18 18:51:37,comment,This looks like a very good solution. If enoug...,6572920.0,6573252,0.000,0.469
7,danmaz74,2013-10-18 10:50:34,comment,"Lost had much higher production value, but B5 ...",6569359.0,6570689,0.074,0.255
5,danmaz74,2013-07-15 13:35:34,comment,"Come on, no serious economist gives credit to ...",6045121.0,6045388,0.056,0.202
1,danmaz74,2012-05-27 14:12:04,comment,"This is what I was afraid of. By the way, with...",4030098.0,4030122,0.140,0.000
2,danmaz74,2011-12-26 18:23:06,comment,"Thank you for your suggestions.The ""Executive ...",3393433.0,3393485,0.036,0.169
9,danmaz74,2011-10-13 09:15:39,comment,In my day - I'm 37 - I remember being VERY fre...,3106543.0,3106671,0.059,0.205
3,danmaz74,2011-09-25 15:11:33,comment,G+ is still missing some form of tags/hashtags...,3035527.0,3035905,0.129,0.089
